In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d  
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from EDA_helper import *

In [3]:
df = pd.read_excel("../datasets/msft_test.xlsx", engine='openpyxl')

df1 = pd.read_csv("../datasets/constituents_csv.csv")
sp_energ = df1[df1["Sector"] == "Energy"]
ratings = pd.read_excel("../datasets/100_ratings_06-17.xlsx")
figs = pd.read_excel("../datasets/fin_figs_1.xlsx")

c:\users\rodri\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
len(ratings['Company Name'].unique())

86

In [3]:
a = list(sp_energ["Symbol"])
b = ["RDS.A", "PTR", "TOT", "SNP", "BP", "ENB", "PBR", "CEO", "EQNR", "EPD", "TRP", "COP","E", "KMI",'PSX', 
 'EOG', 'SLB', 'WMB', 'MPC', 'CNQ', 'EC', 'MPLX', 'SU', 'VLO', 'BKR', 'CQP', 'PXD', 'ET', 'OKE', 'HES', 
 'LNG', 'HAL', 'PBA', 'IMO', 'OXY', 'CXO', 'MMP', 'TS', 'COG', 'PAA', 'PSXP', 'FANG', 'CLR', 'CVE', 
 'WES', 'TRGP', 'HESM', 'PE', 'DVN', 'SSL', 'SHI', 'NOV','TPL','EQT','MRO','PDS','APA','VVV','UGP',
 'NFG','CZZ','SHLX','HFC','CCJ','YPF','WPX','FTI','DCP','XEC','AM','ETRN','OVV','REGI','ENBL','SUN','CNX',
 'CHX', 'TCP','SWN','HP','RRC',
     
    'WHD','INT','EURN','PDCE','ENLC','PAGP','BSM','CVI','MUR','MGY','HEP',
     'NS','DKL','FRO','CRC', 'WHD',  'CZZ',  'TCP',  'SWN',  'INT',  'CLNE',  'RRC',  'CRC',  'MUR',  'ENLC', 
    'CVI',  'MTDR', 'PAGP',  'AR', 'RIG',  'NS',  'DKL',  'CLB',  'EURN',  'BSM', 
    'HEP',  'DEN',  'CPG',  'AROC', 'DK',  'LBRT',  'CEQP',  'RTLR',  'USAC',  'BPMP',  'GLNG', 'FRO', 
     'DRQ',  'BROG', 'PTEN',  'KOS',  'NBLX',  'TGP', 'CRK',  'SM',
    'OH',  'NXE',  'PBF',  'VNOM',  'WLL',  'DHT', 'DNOW', 'NESR',  'CKH',  'PUMP',
     'VTOL',  'RES',  'TELL',  'VET',  'TALO', 'OAS', 'NEX', 'PARR',  'QEP', 'BOOM'] 

## did top 140 energy companies by market cap on finviz

a.extend(b)
tik = set(a)
file = open("tickers_plus.txt",'w')

for l in tik:
    file.write(l)
    file.write('\n')
file.close()


In [4]:
ticks = pd.read_excel("../datasets/energ_tickers.xlsx")

In [9]:
big_ticks = list(ticks['Ticker Symbol'].unique())


In [10]:
file = open("tickers_energ_all.txt",'w')

for l in big_ticks:
    file.write(l)
    file.write('\n')
file.close()

In [5]:
# ratings["S&P Domestic Long Term Issuer Credit Rating"].value_counts()
# # len(figs_1["Global Company Key"].uni
# pd.DataFrame(figs_1.groupby('Ticker Symbol')['Cash'].mean()).describe()
# ratings["S&P Domestic Long Term Issuer Credit Rating"].isna().sum()/len(ratings)


In [6]:
figs_1, ratings1 = format_df(figs, ratings) 

Unique Companies in Ratings:  73
Unique Companies in Features:  95
Intersection of companies:  73


### Delaware is the state of incorporation of nearly 90% of samples companies

In [7]:
# (figs_1["Current State/Province of Incorporation Code"].value_counts()
# /figs_1["Current State/Province of Incorporation Code"].notna().sum())
# figs_1["Current State/Province of Incorporation Code"].unique()

### Ratings Appear to be "Normally" Distributed


However, ratings are discrete, categorical data and make no sense in the context of the normal distribution.
For multi-class classification, we want uniformly distributed labels.

In [ ]:
embed = {'BBB-':-1, 'BBB':0, 'A-':2, 'BBB+':1, 'AA+':7, 'AA':6, 'A':3, 'AA-':5, 'BB':-3, 'BB+':-2,
       'AAA':8, 'B':-6, 'B+':-5, 'A+':4, 'BB-':-4, 'CCC+':-8, 'B-':-7}

fig, ax = plt.subplots(figsize=(7,7))
embedded =  merged['rating'].apply(lambda x: embed[x])
ax.hist(embedded,bins=17)
ax.set_xticklabels([0, 'CCC+', 'B','BB-','BB+', 'BBB','A-','A+',
                   'AA', 'AAA'])

hmean = np.mean(embedded)
hstd = np.std(embedded)
pdf = stats.norm.pdf(embedded, hmean, hstd)

ax2 = ax.twinx()
ax2.plot(embedded, pdf, 'r.')
plt.show()

### Ratings are discrete and can't be understood by a continuous distribution like the Gaussian 

Jarque-Bera Goodness of Fit test checks skewness and kurtosis of distribution to qualify if it is normally distributed


High statistic value and low p-value implies the distribution is NOT Gaussian.

This is obvious since our ratings are categorical data and make no sense in the context of the normal distribution. However, the fact that the labels, when plotted, seem to resemble a normal dist. around BBB may be useful.

In [ ]:
t = np.random.normal(0,1, 100)

print("JB test for Ratings: ", "statistic: ", stats.jarque_bera(embedded)[0], "p-value: ", stats.jarque_bera(embedded)[1])
print("JB test for Std Normal (Control): ", "statistic: ", stats.jarque_bera(t)[0], "p-value: ", stats.jarque_bera(t)[1])

In [ ]:
# energs = pd.read_excel("../datasets/energ_specific_all.xlsx")
# m = pd.to_datetime(energs['Data Date'], infer_datetime_format=True)
# q = m.apply(lambda x: x.strftime('%Y%m%d'))
# energs['Data Date'] = q
# energs.to_excel("../datasets/energ_specific_all_new.xlsx")